# How to use lingtypology

This is a simple tutorial that allows to start using Lingtypology.  
For more information you can consult docstrings (e.g. `lingtypology.LingMap.__doc__`).

## 0. Table of Contents
### 1. [Maps](#maps)
#### 1.1. [Simplest example](#simplest_example)  
#### 1.2. [Features](#features)
#### 1.3. [Map Customization](#map_cust)
#### 1.4. [Accessing Glottolog](#access_glot)
#### 1.5. [Customizing Features and Controls](#feat_cont)
#### 1.6 [Stroke Features](#s_features)
#### 1.7. [Heatmaps](#heatmaps)
#### 1.8.  [Lines](#lines)
### 2. [Real examples](#real_examples)
#### 2.1. [Circassian Example](#example1)
#### 2.2. [Another way](#example2)
#### 2.3. [Consonants Example](#example3)
### 3. [Wals](#wals)
### 4. [Autotyp](#autotyp)
### 5. [Glottolog](#glottolog)
#### 5.1 [Glottolog Usage](#g_usage)
#### 5.2. [Glottolog Version](#g_version)
### 8. [Credits](#credits)
### 9. [Citation](#citation)
### 10. [License](#license)

## 1. Maps <a name="maps"></a>
In this paragraph I will consider some basic features that you can use.

### 1.1. Simplest example <a name="simplest_example"></a>

The simplest script that draws a map with Romanian and Ukrainian languages will be:

In [1]:
import lingtypology

In [2]:
m = lingtypology.LingMap(('Romanian', 'Ukrainian'))
m._create_map()

Be advised that `_create_map()` returns a `Folium` map. In this notebook it is used because the map may
be displayed this way.  
To save your map as html run: `m.save('map.html')`  
To get html as `str` run: `m.render()`

### 1.2. Features <a name="features"></a>
Simply drawing languages on a map is not very interesting. Let's say that we want to mark some morphological features as different colors.

In [3]:
languages = ["Adyghe", "Kabardian", "Polish", "Russian", "Bulgarian"]
features = ["Agglutinative", "Agglutinative", "Inflected", "Inflected", "Analythic"]
m = lingtypology.LingMap(languages)
m.add_features(features)
m._create_map()

### 1.3. Customizing the map <a name="map_cust"></a>
As you can see, the map is not centered properly. It can be fixed with passing coordinates to the `start_location` attribute. You can change start zoom as well `start_zoom`.
Also, you can change colors using the `colors` attribute.

In [4]:
m.start_location = (40, 40)
m.start_zoom = 3
m.colors = ("yellowgreen", "red", "blue")
m._create_map()

### 1.4. Accessing Glottolog <a name="access_glot"></a>

You can also use `lingtypology.glottolog` to get some data from it. For example, I want to add language affiliations to popups.

In [5]:
affs = lingtypology.glottolog.get_affiliations(languages)
m.add_popups(affs)
m._create_map()

### 1.5. Customizing features and controls <a name="feat_cont"></a>

You can pass additional parameters to the `add_features` method.  
If for some reason you do not wish to use colors, you could use shapes.  
If you want to add controls, you can do it as well.

In [6]:
m.add_features(features, use_shapes=True, control=True)
m._create_map()

### 1.6. Stroke features <a name="s_features"></a>

It is possible to add another set of features to strokes of markers.

In [7]:
m = lingtypology.LingMap(languages)
m.add_features(features)
m.add_stroke_features(['Ergative', 'Ergative', 'Accusative', 'Accusative', 'Accusative'])
m._create_map()

### 1.7. Heatmaps <a name="heatmaps"></a>
It would seem unfair if we could only draw circles.  
Let's draw a heatmap (the more density, the more languages with Large consonant inventory).

In [8]:
import lingtypology
#I take coordinates from Wals. More in paragraph 3.Wals
import pandas
from lingtypology.db_apis import Wals
wals_page = Wals('1a').get_df()

#First initialize LingMap without languages
m = lingtypology.LingMap()
#Add heatmap from  the Wals data
m.add_heatmap(wals_page[wals_page._1A == 'Large'].coordinates)
#Let's also add a title to the map
m.title = 'Large Consonant Inventories'
m._create_map()

Citation for feature 1A:
Ian Maddieson. 2013. Consonant Inventories.
In: Dryer, Matthew S. & Haspelmath, Martin (eds.)
The World Atlas of Language Structures Online.
Leipzig: Max Planck Institute for Evolutionary Anthropology.
(Available online at http://wals.info/chapter/1, Accessed on 2019-05-08.)



### 1.8. Lines <a name="lines"></a>

Let's say we have this list of languages:

In [21]:
balkan = ['Modern Greek', 'Romanian', 'Bulgarian', 'Macedonian', 'Gheg Albanian']
other = ['Ukrainian', 'Turkish', 'Italian']
languages = balkan + other

We want the ones from Balkan sprachbund connected with lines.

In [22]:
#Let's get the coordinates that we need to draw a line
coordinates = map(lingtypology.glottolog.get_coordinates, balkan)
m = lingtypology.LingMap(languages)
m.add_line(coordinates)
m.start_location = (43, 27)
m.start_zoom = 5
m._create_map()

## 2. Some real examples <a name="real_examples"></a>

### 2.1. Circassian Example <a name="example1"></a>
Let's draw a map based on data from [this](https://github.com/ropensci/lingtypology/blob/master/database_creation/circassian.csv) CSV.  

In [9]:
import pandas

In [10]:
circassian = pandas.read_csv(
    'https://raw.githubusercontent.com/ropensci/lingtypology/master/database_creation/circassian.csv',
    delimiter=',',
    header=0,
)

In [11]:
coordinates = zip(list(circassian.latitude), list(circassian.longitude))
dialects = circassian.dialect
languages = circassian.language
popups = circassian.village

In [12]:
#Creating LingMap object
m = lingtypology.LingMap(languages)
#Setting up start location
m.start_location = (44.21, 42.32)
#Setting up start zoom
m.start_zoom = 7
#Inner features < dialect
m.add_features(dialects)
#Outer features < language
m.add_stroke_features(languages)
#Popups < village
m.add_popups(popups)
#Tooltips < language
m.add_tooltips(languages)
#Custom coordinates (override the ones from Glottolog)
m.add_custom_coordinates(coordinates)
#Inner legend title and position
m.legend_title = 'Dialects'
m.legend_position = 'bottomright'
#Outer legend title and position
m.stroke_legend_title = 'Languages'
m.stroke_legend_position = 'topright'
m._create_map()

### 2.2. Another way <a name="example2"></a>
If you are used to the R package, you may find this way to program the second example more appealing.  
However, it is not recommended because most of customization is impossible.

In [13]:
coordinates = zip(list(circassian.latitude), list(circassian.longitude))
dialects = circassian.dialect
languages = circassian.language
popups = circassian.village

lingtypology.map_feature(
    languages,
    custom_coordinates = coordinates,
    features = dialects,
    stroke_features = languages,
    popups = popups,
    tooltips = languages,
    legend_title = 'Dialects',
    legend_position = 'bottomright',
    stroke_legend_title = 'Languages',
    stroke_legend_position = 'topright',
    start_zoom = 7,
    start_location = (44.21, 42.32),
    save_html = 'circassian.html',
)

### 2.3. Consonants Example <a name="example3"></a>
Map based on [this](https://raw.githubusercontent.com/ropensci/lingtypology/master/database_creation/ejective_and_n_consonants.csv) data.

In [14]:
data = pandas.read_csv(
    'https://raw.githubusercontent.com/ropensci/lingtypology/master/database_creation/ejective_and_n_consonants.csv',
    delimiter=',',
    header=0,
)

In [15]:
m = lingtypology.LingMap(data.language)
m.legend_title = 'Amount of consonants'
m.add_tooltips(data.consonants)
#If numeric is True, it will look like this
m.add_features(data.consonants, numeric=True)
m._create_map()

## 3. Wals <a name="wals"></a>

It is possible to access Wals data (online) using `lingtypology.db_apis`.

In [16]:
import lingtypology
from lingtypology.db_apis import Wals

wals_page = Wals('1a', '2a').get_df()
wals_page.head()

Citation for feature 1A:
Ian Maddieson. 2013. Consonant Inventories.
In: Dryer, Matthew S. & Haspelmath, Martin (eds.)
The World Atlas of Language Structures Online.
Leipzig: Max Planck Institute for Evolutionary Anthropology.
(Available online at http://wals.info/chapter/1, Accessed on 2019-05-08.)

Citation for feature 2A:
Ian Maddieson. 2013. Vowel Quality Inventories.
In: Dryer, Matthew S. & Haspelmath, Martin (eds.)
The World Atlas of Language Structures Online.
Leipzig: Max Planck Institute for Evolutionary Anthropology.
(Available online at http://wals.info/chapter/2, Accessed on 2019-05-08.)



,wals code,language,genus,family,area,coordinates,_1A,_2A
0,kiw,Kiwai (Southern),Kiwaian,Kiwaian,Phonology,"(-8.0, 143.5)",Small,Average (5-6)
1,xoo,!Xóõ,Tu,Tu,Phonology,"(-24.0, 21.5)",Large,Average (5-6)
2,ani,//Ani,Khoe-Kwadi,Khoe-Kwadi,Phonology,"(-18.9166666667, 21.9166666667)",Large,Average (5-6)
3,abi,Abipón,South Guaicuruan,Guaicuruan,Phonology,"(-29.0, -61.0)",Moderately small,Average (5-6)
4,abk,Abkhaz,Northwest Caucasian,Northwest Caucasian,Phonology,"(43.0833333333, 41.0)",Large,Small (2-4)


In [17]:
m = lingtypology.LingMap(wals_page.language)
m.add_custom_coordinates(wals_page.coordinates)
m.add_features(wals_page._1A)
m.legend_title = 'Consonant Inventory'
m._create_map()

## 4. Autotyp <a name="autotyp"></a>

It is possible to access Autotyp data (online) using `lingtypology.db_apis`.

In [18]:
import lingtypology
from lingtypology.db_apis import Autotyp
Autotyp_table = Autotyp('Gender').get_df()
Autotyp_table.head()

Bickel, Balthasar, Johanna Nichols, Taras Zakharko,
Alena Witzlack-Makarevich, Kristine Hildebrandt, Michael Rießler,
Lennart Bierkandt, Fernando Zúñiga & John B. Lowe.
2017. The AUTOTYP typological databases.
Version 0.1.0 https://github.com/autotyp/autotyp-data/tree/0.1.0
(get_by_glot_id) Warning: language by ngiy1239 not found
(get_by_glot_id) Warning: language by east2283 not found


,Language,LID,Gender.n,Gender.binned4,Gender.Presence
0,Godoberi,1531,3,3 genders,True
1,Bininj Kun-Wok,655,4,4 genders,True
2,Luvale,553,10,more than 4 genders,True
3,Chamalal,1149,5,more than 4 genders,True
4,North-Central Dargwa,2949,3,3 genders,True


In [20]:
m = lingtypology.LingMap(Autotyp_table.Language)
m.add_features(Autotyp_table['Gender.binned4'])
m._create_map()

## 5. Glottolog <a name="glottolog"></a>
You can use a number of functions to work with Glottolog data.

### 5.1. Usage <a name="g_usage"></a>

In [ ]:
from lingtypology import glottolog

Get genealogy information:

In [ ]:
glottolog.get_affiliations(('Russian', 'English'))

Get coordinates of a language:

In [ ]:
glottolog.get_coordinates('Russian')

Get the Glottolog ID (aka Glottocode):

In [ ]:
glottolog.get_glot_id('Russian')

Get the macroarea:

In [ ]:
glottolog.get_macro_area('Russian')

Get the ISO code:

In [ ]:
glottolog.get_iso('Russian')

Get language by ISO code:

In [ ]:
glottolog.get_by_iso('rus')

Get language by Glottocode:

In [ ]:
glottolog.get_by_glot_id('russ1263')

Get Glottocode by ISO:

In [ ]:
glottolog.get_glot_id_by_iso('rus')

Get ISO by Glottocode:

In [ ]:
glottolog.get_iso_by_glot_id('russ1263')

### 5.2. Using the most recent version of Glottolog database <a name="g_version"></a>

Each new release of `lingtypology` is shipped with the latest version of the Glottolog data. The information about the version of Glottolog can be retrieved using:

In [ ]:
from lingtypology import glottolog
#Yet, I cannot guarantee that this tutorial is updated with each release.
#If you want to get your version, run this code on your machine.
glottolog.version

Nevertheless, if you want to get the newest release and do not wish to wait for the next version of `lingtypology`, you can download the version you wish locally by following these steps: 

1) Download `glottolog` from [here](https://github.com/clld/glottolog).  

2) If you downloaded it as an archive, unpack it.  

3) In your home directory make a folder and call it `.lingtypology_data` (with `.` in the beginning).  

4) Move `glottolog` into `.lingtypology_data`.  

5) Run the following command:  
`glottolog --repos=glottolog languoids`  

6) It will generate two small files (`csv` and `json`). Now you can delete everything **except** for these files from the directory.

## 8. Credits <a name="credits"></a>

This Python package is inspired by the [R package](https://github.com/ropensci/lingtypology) made by [agricolamz](https://github.com/agricolamz).  
Also this module uses JavaScript library [Leaflet](https://leafletjs.com/) and its Python-wrapper [Folium](https://github.com/python-visualization/folium) and [Branca](https://github.com/python-visualization/branca).  
Also, special thanks to [Xrotwang](https://github.com/xrotwang) for helping me with the package.

## 9. Citation <a name="citation"></a>

It will be most appreciated if you cite `lingtypology`:
```
@misc{michael_voronov_2019_2669684,
author       = {Michael Voronov},
title        = {{lingtypology: a Python tool for linguistic interactive mapping}},
month        = may,
year         = 2019,
doi          = {10.5281/zenodo.2669684},
url          = {https://doi.org/10.5281/zenodo.2669684}
}
```


## 10. License <a name="license"></a>

Copyright (C) 2018-2019  Michael Voronov

This program is free software: you can redistribute it and/or modify  
it under the terms of the GNU General Public License as published by  
the Free Software Foundation, either version 3 of the License, or  
(at your option) any later version.

This program is distributed in the hope that it will be useful,  
but WITHOUT ANY WARRANTY; without even the implied warranty of  
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the  
[GNU General Public License](https://github.com/OneAdder/lingtypology/blob/master/LICENSE.md) for more details.